## Latent Dirichlet Allocation (LDiA)

Latent Sematic Analysis (LSA) should be the first choice for most topic modelling, semantic search, or content-based recommendation engines.
<br>
Given the maths used for LSA is efficient and somewhat easy to follow, given that it produces a linear transformation that can be applied to new batches of natural language without training and little loss in accuracy. However, LDiA can give slightly better results in some situations. 

LDiA performs many things accomplished by topic models with LSA (and SVD under the hood), though unlike LSA, LDiA assumes a Dirichlet distribution of word frequencies. It's more precise about the statistics of allocating words to topics than the linear math of LSA.


LDiA creates a semantic vector space model (as with topic vectors) using a topic mix process that for any given document, such topics can be determined by the word mixtures in each topic by which topic those words were assigned to.
<br>
In some ways, this makes an LDiA topic model easier to understand, as words assigned to topics and topics assigned to documents tend to make more sense than for LSA.
<br>
LDiA assumes that each document is a mixture (linear combination) of some arbitrary number of topics that one can select when beginning to train the LDiA model. LDiA also assumes that each topic can be represented by a distribution of words (term frequencies). 
<br>
The probability/weight for each of these topics within a document, along with the probability of a word being assigned to a topic, is assumed to begin with a Dirichlet probability distribution (*prior* in statistics), where the algorithm derives from.